# CONNECT TO DRIVE AND IMPORTS

In [6]:
!pip install SimpleITK


from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os
import sys
import inspect
import numpy as np
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt

import SimpleITK as sitk

from google.colab.patches import cv2_imshow

In [8]:
path = "/content/drive/Shareddrives/IA DL_project/ML IA/nb_Aron"

if path not in sys.path:
  sys.path.append(path)

import luna_module
from luna_module import *

# List all function names in the luna_module
function_names = [name for name, obj in inspect.getmembers(luna_module) if inspect.isfunction(obj)]
print(function_names)

['annotations_by_uid', 'binarize_lung', 'binarize_lung_3d', 'binary_closing', 'binary_dilation', 'binary_erosion', 'binary_fill_holes', 'binary_opening', 'center_of_mass', 'clear_border', 'convert_annotation_df', 'convert_annotation_df_with_uid', 'create_3d_mask', 'create_annotations_mask', 'create_annotations_mask_by_uid', 'create_patch', 'debugger', 'draw_ellipsoid', 'find_by_uid', 'find_neighborhood_indices', 'find_neighborhood_indices_more_precise', 'get_slice_candidates', 'get_slice_candidates_old', 'get_slices', 'get_uids', 'img_by_uid', 'masked_annotations_by_uid', 'masked_annotations_with_info_by_uid', 'meta_by_uid', 'norm2float', 'norm2uint16', 'norm2uint8', 'normalize_intensity', 'plot_slices', 'process_slice_candidates', 'process_slice_candidates_old', 'process_slices', 'remove_non_central_objects', 'sensitivity_score', 'sensitivity_score_more_precise', 'show_3_images', 'subset_by_uid', 'unwanted_object_filter']


# CONSTANTS

In [9]:
LUNA_PATH = os.path.join(os.getcwd(), "drive", "Shareddrives", "IA DL_project", "ML IA", "LUNA16")

SUBSET = "subset9"

SUBSETS_PATH = os.path.join(LUNA_PATH, "subsets") # path for subsets folder
SUBSET_PATH =  os.path.join(SUBSETS_PATH, SUBSET) # path for subsets folder
SAVE_FOLDER_PATH = os.path.join(LUNA_PATH, "feature_extractions", "nomask_noclip_gabor_4haar")


ANNOTATIONS_DF = pd.read_csv(os.path.join(LUNA_PATH, f"{SUBSET}_annotations_expanded.csv"), index_col="Unnamed: 0")

SUBSETS = os.listdir(SUBSET_PATH) # subset folders present
FILENAMES = os.listdir(SUBSET_PATH)
UIDS = set(map(lambda filename: os.path.splitext(filename)[0], FILENAMES))

# EXTRACT FEATURES

## HAAR FEATURES

In [10]:
import numpy as np
import cv2 as cv
import pandas as pd

def create_center_surround_kernel(size, inner_size):
    kernel = np.zeros((size, size), dtype=np.float32)

    # Define the start and end points of the inner region
    start = (size - inner_size) // 2
    end = start + inner_size

    # Set the central region to +1
    kernel[start:end, start:end] = 1

    # Set the surround region to -1
    kernel[:start, :] = -1
    kernel[end:, :] = -1
    kernel[:, :start] = -1
    kernel[:, end:] = -1

    return kernel

def create_four_rectangle_kernel(size):
    kernel = np.zeros((size, size), dtype=np.float32)

    half_size = size // 2

    # Top-left and bottom-right rectangles (+1)
    kernel[:half_size, :half_size] = 1
    kernel[half_size:, half_size:] = 1

    # Top-right and bottom-left rectangles (-1)
    kernel[:half_size, half_size:] = -1
    kernel[half_size:, :half_size] = -1

    return kernel

def extract_haar_features(p_haar, kernel):
    # Ensure the patch and kernel are float32
    p_haar = p_haar.astype(np.float32)
    kernel = kernel.astype(np.float32)

    # Apply the kernel to the patch using convolution
    filtered_patch = cv.filter2D(p_haar, -1, kernel)
    return filtered_patch

def multi_scale_haar_features(p_haar, kernel, scales):
    haar_features_list = []

    for scale in scales:
        # Resize the patch
        p_haar = p_haar.astype('float32')

        row_th = int(p_haar.shape[0]*scale)
        col_th = int(p_haar.shape[1]*scale)

        scaled_patch = cv.resize(p_haar, (row_th, col_th))
        # scaled_patch = cv.resize(p_haar, (row_th, col_th), fx=scale, fy=scale, interpolation=cv.INTER_LINEAR)

        # Ensure the scaled patch is valid
        if scaled_patch.size == 0:
            raise ValueError(f"Scaled patch is empty at scale {scale}")

        # Extract Haar features from the scaled patch
        haar_features = extract_haar_features(scaled_patch, kernel)
        haar_features_list.append(haar_features)

        # Find the location of the maximum response
        min_val, max_val, min_loc, max_loc = cv.minMaxLoc(haar_features)

    return haar_features_list


# Function to pad a feature map to a specified size
def pad_feature_map(feature_map, target_size):
    padded_feature_map = np.zeros((target_size, target_size), dtype=np.float32)
    current_size = feature_map.shape[0]

    # Calculate padding offsets
    offset = (target_size - current_size) // 2

    # Place the feature map in the center of the padded array
    padded_feature_map[offset:offset + current_size, offset:offset + current_size] = feature_map

    return padded_feature_map

def combine_and_average_feature_maps(feature_maps, target_size):
    padded_feature_maps = [pad_feature_map(fm, target_size) for fm in feature_maps]
    combined_feature_map = np.mean(padded_feature_maps, axis=0)

    return combined_feature_map

def combine_and_weighted_average_feature_maps(feature_maps, target_size, weights):
    weighted_sum = np.zeros((target_size, target_size), dtype=np.float32)
    total_weight = 0

    for feature_map, weight in zip(feature_maps, weights):
        padded_feature_map = pad_feature_map(feature_map, target_size)
        weighted_sum += padded_feature_map * weight
        total_weight += weight

    combined_feature_map = weighted_sum / total_weight
    return combined_feature_map

def custom_pooling(feature_map, kernel_size, pooling_type='avg'):
    h, w = feature_map.shape
    pooled_map = np.zeros((h // kernel_size, w // kernel_size), dtype=np.float32)

    for i in range(0, h, kernel_size):
        for j in range(0, w, kernel_size):
            patch = feature_map[i:i + kernel_size, j:j + kernel_size]
            if pooling_type == 'avg':
                pooled_map[i // kernel_size, j // kernel_size] = np.mean(patch)
            elif pooling_type == 'max':
                pooled_map[i // kernel_size, j // kernel_size] = np.max(patch)
            else:
                raise ValueError("Pooling type must be 'avg' or 'max'")

    return pooled_map

def haar_pooled(patch, kerneltype, target_size = 48):
    """
    Perform multi-scale Haar feature extraction and pooling on the input patch.

    Args:
        p_haar (np.ndarray): The input patch.
        kerneltype (str): Type of Haar kernel ('cs' for center-surround, 'fr' for four-rectangle).
        target_size (int): Desired size of patch rectangle

    Returns:
        tuple: Pooled feature maps (average pooling, max pooling).
    """
    # Initialize Haar parameters
    kernel_size_cs = 8
    inner_size_cs = 4
    size_fr = 8

    # Define scales based on patch size
    scales = [1, 0.875, 0.75, 0.625, 0.5, 0.375, 0.25, 0.125] if patch.shape[0] == 64 else [1, 0.875, 0.75, 0.625, 0.5, 0.375, 0.25]

    # Calculate weights for the scales
    weights = [1 / scale for scale in scales]
    weights = [w / sum(weights) for w in weights]

    # Pooling parameters
    kernel_poolsize = 8

    # Select the appropriate kernel
    if kerneltype == 'cs':
        kernel = create_center_surround_kernel(kernel_size_cs, inner_size_cs)
    elif kerneltype == 'fr':
        kernel = create_four_rectangle_kernel(size_fr)
    else:
        raise ValueError("Kernel type must be 'cs' (center-surround) or 'fr' (four-rectangle)")

    # Extract Haar features at multiple scales
    haar_features_list = multi_scale_haar_features(patch, kernel, scales)

    # Combine and weight the feature maps
    combined_feature_map = combine_and_weighted_average_feature_maps(haar_features_list, target_size, weights)
    # combined_feature_map = combine_and_average_feature_maps(haar_features_list, target_size)

    # Perform custom pooling
    pooled_feature_map_avg = custom_pooling(combined_feature_map, kernel_poolsize, pooling_type='avg')
    pooled_feature_map_max = custom_pooling(combined_feature_map, kernel_poolsize, pooling_type='max')

    return pooled_feature_map_avg, pooled_feature_map_max


# Example usage
# p_haar = #your patch
# pooled_avg, pooled_max = haar_pooled(p_haar, 'cs')


## GABOR FEATURES

In [11]:
def create_gabor_kernels(ksize=32, sigma=4.0, lambd=10.0, gamma=0.5, psi=0):
    """
    Create a set of Gabor kernels with different orientations and frequencies.

    Args:
        ksize (int): Size of the Gabor kernel.
        sigma (float): Standard deviation of the Gaussian function.
        lambd (float): Wavelength of the sinusoidal factor.
        gamma (float): Spatial aspect ratio.
        psi (float): Phase offset.

    Returns:
        list: A list of Gabor kernels.
    """
    kernels = []
    for theta in np.arange(0, np.pi, np.pi / 8):  # 8 orientations
        kernel = cv.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi, ktype=cv.CV_32F)
        kernels.append(kernel)
    return kernels

def apply_gabor_kernels(patch, kernels):
    """
    Apply Gabor kernels to a patch to extract features.

    Args:
        patch (np.ndarray): The input image patch (64x64).
        kernels (list): A list of Gabor kernels.

    Returns:
        np.ndarray: The combined Gabor feature vector.
    """
    features = []
    for kernel in kernels:
        filtered_patch = cv.filter2D(patch, cv.CV_32F, kernel)
        features.append(filtered_patch)

    # Combine features (e.g., by flattening and concatenating)
    combined_features = np.hstack([f.flatten() for f in features])
    return combined_features

def apply_gabor_kernels_and_compute_mean(patch, kernels):
    """
    Apply Gabor kernels to a patch and compute the mean feature map.

    Args:
        patch (np.ndarray): The input image patch (64x64).
        kernels (list): A list of Gabor kernels.

    Returns:
        np.ndarray: The mean Gabor feature map.
    """
    feature_maps = []
    for kernel in kernels:
        filtered_patch = cv.filter2D(patch, cv.CV_32F, kernel)
        feature_maps.append(filtered_patch)

    # Compute the mean feature map
    mean_feature_map = np.mean(feature_maps, axis=0)
    return mean_feature_map

def custom_pooling(feature_map, kernel_size, pooling_type='avg'):
    h, w = feature_map.shape
    pooled_map = np.zeros((h // kernel_size, w // kernel_size), dtype=np.float32)

    for i in range(0, h, kernel_size):
        for j in range(0, w, kernel_size):
            patch = feature_map[i:i + kernel_size, j:j + kernel_size]
            if pooling_type == 'avg':
                pooled_map[i // kernel_size, j // kernel_size] = np.mean(patch)
            elif pooling_type == 'max':
                pooled_map[i // kernel_size, j // kernel_size] = np.max(patch)
            else:
                raise ValueError("Pooling type must be 'avg' or 'max'")

    return pooled_map


def gabor_pooled(patch, kernel_poolsize=8):
    """
    Apply Gabor filters to the patch, compute mean feature map, and perform pooling.

    Args:
        patch (np.ndarray): The input image patch (64x64).
        kernel_poolsize (int): Size of the pooling kernel.

    Returns:
        tuple: Pooled feature maps (average pooling, max pooling).
    """
    # Create Gabor kernels
    kernels = create_gabor_kernels()

    # Apply Gabor kernels and compute the mean feature map
    mean_feature_map = apply_gabor_kernels_and_compute_mean(patch, kernels)

    # Perform custom pooling
    pooled_feature_map_avg = custom_pooling(mean_feature_map, kernel_poolsize, pooling_type='avg')
    pooled_feature_map_max = custom_pooling(mean_feature_map, kernel_poolsize, pooling_type='max')

    return pooled_feature_map_avg, pooled_feature_map_max

## FEATURE EXTRACT FUNCTION

In [12]:
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from skimage.measure import regionprops, label
from skimage import io, color

def debugger(img, title=None):
    if title is not None:
        print(title)
    print(np.unique(img))
    plt.imshow(img, cmap="gray")
    plt.show()

def mask_closest_object_to_center(binary_image, debug=False):
    def debugger(image, step):
        # Debug function placeholder; replace with actual debug functionality if needed
        print(f"{step}: {image}")

    if debug:
        debugger(binary_image, "mask_closest_object_to_center 1st step")

    # Label connected components
    labeled_image = label(binary_image)

    if debug:
        debugger(labeled_image, "mask_closest_object_to_center 2nd step")

    # Get image center
    image_center = tuple(np.array(binary_image.shape) // 2)

    # Check if the center pixel is within any labeled object
    center_label = labeled_image[image_center]
    if center_label > 0:
        # Create a mask for the object containing the center pixel
        center_mask = np.zeros_like(binary_image, dtype=bool)
        center_mask[labeled_image == center_label] = 1

        if debug:
            debugger(center_mask, "mask_closest_object_to_center - center object found")

        center_mask = binary_fill_holes(center_mask)
        center_mask = binary_dilation(center_mask, morphology.disk(1))

        return center_mask.astype(float)

    region_properties = regionprops(labeled_image)
    if not len(region_properties):
        raise Exception("No labels were found")

    # Initialize variables to track the closest object
    min_distance = float('inf')
    closest_label = None

    # Iterate through each labeled object
    for region in region_properties:
        # Compute the centroid of the object
        centroid = region.centroid

        # Calculate the Euclidean distance to the image center
        distance = np.linalg.norm(np.array(centroid) - image_center)

        # Update the closest object if this one is closer
        if distance < min_distance:
            min_distance = distance
            closest_label = region.label

    # Create a mask for the closest object
    closest_mask = np.zeros_like(binary_image, dtype=bool)
    closest_mask[labeled_image == closest_label] = 1

    if debug:
        debugger(closest_mask, "mask_closest_object_to_center 3rd step")

    closest_mask = binary_fill_holes(closest_mask)
    closest_mask = binary_dilation(closest_mask, morphology.disk(1))

    if debug:
        debugger(closest_mask, "mask_closest_object_to_center 4th step")

    return closest_mask.astype(float)

from skimage.feature import hog
from skimage import exposure

def extract_hog_features(patch):
    """
    Extract Histogram of Oriented Gradients (HOG) features from a given patch.

    Args:
        patch (np.ndarray): The input image patch (64x64).

    Returns:
        tuple: HOG features and HOG image.
    """
    # Ensure the patch is of type float32
    patch = patch.astype(np.float32)

    # Compute HOG features
    hog_features = hog(patch,
                        orientations=9,
                        pixels_per_cell=(8, 8),
                        cells_per_block=(2, 2),
                        visualize=False)

    return hog_features

def extract_features(nodule_roi, roi_mask, thresholding="otsu", clip=False, verbose=False, debug=False):

  if debug:
    debugger(nodule_roi, "nodule_roi")
    debugger(roi_mask, "roi_mask")


  if clip:
    n_th = -200
    p_th = 200

    nodule_roi[nodule_roi<n_th] = AIR_TH
    nodule_roi[nodule_roi>p_th] = AIR_TH
    if len(nodule_roi[nodule_roi!=AIR_TH].nonzero()[0]) < 3: # >3 because if there is only 2 nonzero pix it should discard
      raise Exception("After clipping no information was left")

  # roi_mask = binary_dilation(roi_mask, morphology.disk(1))
  # if debug:
  #   debugger(roi_mask, "roi_mask")



  ret, otsu_img = cv.threshold(norm2uint8(nodule_roi), 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
  ret, binary_th_img = cv.threshold(norm2uint8(nodule_roi), 127, 255, cv.THRESH_BINARY)

  match (thresholding):
    case "otsu":
      th = otsu_img
    case "binary":
      th = binary_th_img
    case _:
      th = otsu_img

  if debug:
    debugger(th, "thresholded")

  th[roi_mask == False] = 0

  if debug:
    debugger(th, "thresholded+masked")


  center_object_mask = mask_closest_object_to_center(th, debug)
  if debug:
    debugger(center_object_mask, "center_object_mask")

  labeled_nodule = label(center_object_mask)
  if debug:
    debugger(labeled_nodule, "labeled_nodule")

  props = regionprops(labeled_nodule)[0]
  if debug:
    print(props)

  # Binary Shape features
  area = props.area
  if debug:
    print(f"area: {area}")

  perimeter = props.perimeter
  if debug:
    print(f"perimeter: {perimeter}")

  compactness = (perimeter ** 2) / area
  if debug:
    print(f"compactness: {compactness}")

  eccentricity = props.eccentricity
  if debug:
    print(f"eccentricity: {eccentricity}")

  major_axis_length = props.major_axis_length
  if debug:
    print(f"major_axis_length: {major_axis_length}")

  minor_axis_length = props.minor_axis_length
  if debug:
    print(f"minor_axis_length: {minor_axis_length}")

  solidity = props.solidity
  if debug:
    print(f"solidity: {solidity}")

  extent = props.extent
  if debug:
    print(f"extent: {extent}")

  # Texture features using GLCM
  nodule_roi_uint8 = norm2uint8(nodule_roi)
  glcm = graycomatrix(nodule_roi_uint8, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)

  contrast = graycoprops(glcm, 'contrast')[0, 0]
  if debug:
    print(f"contrast: {contrast}")

  correlation = graycoprops(glcm, 'correlation')[0, 0]
  if debug:
    print(f"correlation: {correlation}")

  energy = graycoprops(glcm, 'energy')[0, 0]
  if debug:
    print(f"energy: {energy}")

  homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
  if debug:
    print(f"homogeneity: {homogeneity}")

  # Intensity features
  mean_intensity = np.mean(nodule_roi)
  if debug:
    print(f"mean_intensity: {mean_intensity}")

  std_intensity = np.std(nodule_roi)
  if debug:
    print(f"std_intensity: {std_intensity}")

  skewness = np.mean(((nodule_roi - mean_intensity) / std_intensity) ** 3)
  if debug:
    print(f"skewness: {skewness}")

  kurtosis = np.mean(((nodule_roi - mean_intensity) / std_intensity) ** 4) - 3
  if debug:
    print(f"kurtosis: {kurtosis}")

  # LBP texture feature
  lbp = local_binary_pattern(nodule_roi, P=8, R=1, method='uniform')
  if debug:
    print(f"lbp: {lbp}")

  lbp_hist, _ = np.histogram(lbp, bins=np.arange(0, 10), range=(0, 9))
  lbp_hist = lbp_hist / np.sum(lbp_hist)
  if debug:
    print(f"lbp_hist: {lbp_hist}")

  # Haar features
  # Center sorround
  pooled_avg_cs, pooled_max_cs = haar_pooled(nodule_roi, 'cs')
  if debug:
    print(f"pooled_avg_cs: {pooled_avg_cs}")
    print(f"pooled_max_cs: {pooled_max_cs}")
  # hog = extract_hog_features(nodule_roi)

  # Four rectangle
  pooled_avg_fr, pooled_max_fr = haar_pooled(nodule_roi, 'fr')
  if debug:
    print(f"pooled_avg_fr: {pooled_avg_fr}")
    print(f"pooled_max_fr: {pooled_max_fr}")

  # Gabor features
  gabor_features_avg, gabor_features_max = gabor_pooled(nodule_roi)
  if debug:
    print(f"gabor_features_avg: {gabor_features_avg}")
    print(f"gabor_features_max: {gabor_features_max}")

  # hog_features = hog(nodule_roi)

  # Aggregate features into a dictionary or array
  feature_dict = {
      'area': area,
      'perimeter': perimeter,
      'compactness': compactness,
      'eccentricity': eccentricity,
      'major_axis_length': major_axis_length,
      'minor_axis_length': minor_axis_length,
      'solidity': solidity,
      'extent': extent,
      'contrast': contrast,
      'correlation': correlation,
      'energy': energy,
      'homogeneity': homogeneity,
      'mean_intensity': mean_intensity,
      'std_intensity': std_intensity,
      'skewness': skewness,
      'kurtosis': kurtosis,
      'lbp_hist': lbp_hist.tolist(),
      "haar_avg_cs": pooled_avg_cs.tolist(),
      "haar_max_cs": pooled_max_cs.tolist(),
      "haar_avg_fr": pooled_avg_fr.tolist(),
      "haar_max_fr": pooled_max_fr.tolist(),
      "pooled_avg_cs": pooled_avg_cs.tolist(),
      "pooled_max_cs": pooled_max_cs.tolist(),
      "gabor_features_avg": gabor_features_avg.tolist(),
      "gabor_features_max": gabor_features_max.tolist(),


      # 'hog': hog_features
  }


  # Unraveling/unpacking keys with lists in them
  expanded_features = {}

  # Iterate over the original dictionary
  for key, value in feature_dict.items():
      if isinstance(value, list) and all(isinstance(i, list) for i in value):
          # If the value is a list of lists, unpack the sublists into new keys
          for i, sublist in enumerate(value, start=1):
              for j, v in enumerate(sublist, start=1):
                  new_key = f"{key}{i}{j}"
                  expanded_features[new_key] = v
      elif isinstance(value, list):
          # If the value is a list, unpack the list into new keys
          for i, v in enumerate(value, start=1):
              new_key = f"{key}{i}"
              expanded_features[new_key] = v
      else:
          # If the value is not a list of lists, keep the original key-value pair
          expanded_features[key] = value

  return expanded_features

## VALIDATION

In [13]:
import pandas as pd
import numpy as np

def convert_annotation_df(annotations_df, verbose=False):
    node_coords = annotations_df["cartesian_coords(zyx)"].apply(
        lambda x: np.array(x.replace("(", "").replace(")", "").split(",")).astype(int)[::-1]
    )
    node_coords = np.array(node_coords.to_list()).reshape((len(node_coords.to_list()), 3))

    # Create a new DataFrame to store the extracted coordinates
    node_coords_df = pd.DataFrame(columns=["x", "y", "z"], index=annotations_df.index)
    node_coords_df["x"] = node_coords[:, 0]
    node_coords_df["y"] = node_coords[:, 1]
    node_coords_df["z"] = node_coords[:, 2]

    if verbose:
      print(f"[ANNOTATIONS] - simplified")
      print(node_coords_df)

    return node_coords_df

def find_neighborhood_indices(large_df, small_df, threshold=15, verbose=False):
    """
    Find indices in the larger DataFrame where the coordinates are within a
    specified neighborhood distance from any row coordinates in the smaller DataFrame.

    Parameters:
    large_df (pd.DataFrame): The larger DataFrame with 'x', 'y', 'z' columns.
    small_df (pd.DataFrame): The smaller DataFrame with 'x', 'y', 'z' columns.
    threshold (float): The neighborhood distance threshold. Default is 15.

    Returns:
    np.ndarray: An array of indices from the larger DataFrame that are within the neighborhood.
    """
    # Initialize an empty list to store the indices
    neighborhood_indices = []
    neighborhood_indices_dict= {}

    if not len(small_df):
      neighborhood_indices, neighborhood_indices_dict

    if verbose:
        print(f"[FIND HITS] - Annotations to find:")
        print(small_df)

    # Iterate through each row in the smaller DataFrame
    for i, row in small_df.iterrows():
        if verbose:
          print(f"Annotation with index {i}:")
          print(row)

        neighborhood_indices_dict[i] = []
        # Create conditions to check for vicinity in all three axes
        a_x = large_df["x"] >= (row["x"] - threshold)
        b_x = large_df["x"] <= (row["x"] + threshold)
        a_y = large_df["y"] >= (row["y"] - threshold)
        b_y = large_df["y"] <= (row["y"] + threshold)
        a_z = large_df["z"] >= (row["z"] - threshold)
        b_z = large_df["z"] <= (row["z"] + threshold)

        # Combine conditions for all three axes
        vicinity_condition = a_x & b_x & a_y & b_y & a_z & b_z

        # Find indices where all conditions are satisfied
        close_indices = large_df.index[vicinity_condition].tolist()

        # Append these indices to the neighborhood_indices list
        neighborhood_indices.extend(close_indices)
        neighborhood_indices_dict[i] = (close_indices)

    # Return unique indices as a numpy array
    return np.unique(neighborhood_indices), neighborhood_indices_dict


def sensitivity_score(candidates_df, annotations_df, threshold=15, verbose=False):
    """
    Calculate the sensitivity score based on candidate detections and annotated coordinates.

    Parameters:
    candidates_df (pd.DataFrame): DataFrame containing candidate coordinates with 'x', 'y', 'z' columns.
    annotations_df (pd.DataFrame): DataFrame containing annotated coordinates in the format with 'x', 'y', 'z' columns.

    Returns:
    float: Sensitivity score, calculated as the ratio of correctly identified annotations to the total number of annotations.
    """
    if not len(annotations_df):
      if verbose:
        print(f"[SENSITIVITY] - No annotations for case -> no sensitivity needed")
      return 0

    # Find neighborhood indices within a threshold distance
    candidate_indices, candidate_dict = find_neighborhood_indices(candidates_df, annotations_df, threshold)

    # Initialize a score counter
    score = 0

    # Iterate over the candidate dictionary to calculate the score
    for key, val in candidate_dict.items():
        if len(candidate_dict[key]): # If there is a candidate in the vicinity for the annotation, it counts as a 'hit'
            score += 1

    # Return the sensitivity score
    return score / len(annotations_df)

In [14]:
convert_annotation_df(annotations_by_uid("1.3.6.1.4.1.14519.5.2.1.6279.6001.187451715205085403623595258748", ANNOTATIONS_DF), verbose=True)

[ANNOTATIONS] - simplified
Empty DataFrame
Columns: [x, y, z]
Index: []


,x,y,z


# RUN FOR ONE

In [29]:
ONE_EXAMPLE_TEST = True

## LOAD EXAMPLE

In [30]:
uid = "1.3.6.1.4.1.14519.5.2.1.6279.6001.187451715205085403623595258748" # analyzed case uid

## CANDIDATES FOR UID

In [31]:
def get_candidates_for_uid(uid, subsets_path, img_3d=None, annotations_df=None, verbose=False, debug=False):
  subset = None
  for subset_dir in os.listdir(subsets_path):
    filenames = os.listdir(os.path.join(subsets_path, subset_dir)) # filenames in subset directory
    filenames = list(map(lambda x: os.path.splitext(x)[0], filenames)) # filename w/o extension
    filenames = set(filenames) # unique filenames (because .mhd and .raw have same filename)
    if uid in filenames:
      subset = subset_dir
      break # if found -> stop

  if subset is None:
    raise FileNotFoundError("No image under this uid")

  if img_3d is None:
    img_path = os.path.join(subsets_path, subset, f"{uid}.mhd") # .mhd path
    mhd_img = sitk.ReadImage(img_path) # SimpleITK object
    img_3d = sitk.GetArrayFromImage(mhd_img)
    img_3d[img_3d < AIR_TH] = AIR_TH

  candidates_df, candidate_masks = process_slice_candidates(img_3d, verbose=verbose, debug=debug)

  return candidates_df

In [32]:
if ONE_EXAMPLE_TEST:
  candidates_df = get_candidates_for_uid(uid, SUBSETS_PATH, annotations_df=ANNOTATIONS_DF, verbose=True)

[START] - processing slice #0
[DONE] - 0 candidates found for slice #0

[STATUS] - 0 candidates found
[STATUS] - 125 slices left

[START] - processing slice #1
[DONE] - 0 candidates found for slice #1

[STATUS] - 0 candidates found
[STATUS] - 124 slices left

[START] - processing slice #2
[DONE] - 0 candidates found for slice #2

[STATUS] - 0 candidates found
[STATUS] - 123 slices left

[START] - processing slice #3
[DONE] - 0 candidates found for slice #3

[STATUS] - 0 candidates found
[STATUS] - 122 slices left

[START] - processing slice #4
[DONE] - 0 candidates found for slice #4

[STATUS] - 0 candidates found
[STATUS] - 121 slices left

[START] - processing slice #5
[DONE] - 0 candidates found for slice #5

[STATUS] - 0 candidates found
[STATUS] - 120 slices left

[START] - processing slice #6
[DONE] - 0 candidates found for slice #6

[STATUS] - 0 candidates found
[STATUS] - 119 slices left

[START] - processing slice #7
[DONE] - 0 candidates found for slice #7

[STATUS] - 0 candi

## EXTRACT FEATURES FOR UID

In [33]:
def extract_features_for_uid(uid, subsets_path, candidates_df, img_3d=None, verbose=False, debug=False):
  subset = None
  for subset_dir in os.listdir(subsets_path):
    filenames = os.listdir(os.path.join(subsets_path, subset_dir)) # filenames in subset directory
    filenames = list(map(lambda x: os.path.splitext(x)[0], filenames)) # filename w/o extension
    filenames = set(filenames) # unique filenames (because .mhd and .raw have same filename)
    if uid in filenames:
      subset = subset_dir
      break # if found -> stop

  if subset is None:
    raise FileNotFoundError("No image under this uid")

  if img_3d is None:
    img_path = os.path.join(subsets_path, subset, f"{uid}.mhd") # .mhd path
    mhd_img = sitk.ReadImage(img_path) # SimpleITK object

    img_3d = sitk.GetArrayFromImage(mhd_img)
    img_3d[img_3d < AIR_TH] = AIR_TH

  mask_3d = binarize_lung_3d(img_3d).astype(bool)


  # EXTRACT FEATURES
  candidate_features_df = pd.DataFrame()

  for i, row in candidates_df.iterrows():
    if verbose:
      print(f"[FEATURES] - {uid} - extracting features for candidate #{i}")

    slice_ = img_3d[row["z"],:,:] # get slice for candidate
    candidate_roi, _ = create_patch(slice_, (row["y"],row["x"]))

    slice_mask = mask_3d[row["z"],:,:] # get lung mask for slice
    candidate_roi_mask, _ = create_patch(slice_mask, (row["y"],row["x"]))
    if debug:
      debugger(candidate_roi, "candidate_roi")

    try:
      fts = extract_features(candidate_roi, candidate_roi_mask, debug=debug)
      if verbose:
        print(f"Features:")
        print(fts)
      features_df = pd.DataFrame([fts], index=[i])
      candidate_features_df = pd.concat([candidate_features_df, features_df])
    except Exception as e: # If feature extraction fails (likely empty image) remove the candidate
      if verbose:
        print(f"[ERROR] - {e}")
        print(f"[INFO] - dropping candidate #{i}")
      candidates_df = candidates_df.drop(index=i)
      continue


  return candidate_features_df

In [ ]:
if ONE_EXAMPLE_TEST:
  features_df = extract_features_for_uid(uid, SUBSETS_PATH, candidates_df, verbose=True, debug=False)

In [ ]:
if ONE_EXAMPLE_TEST:
  features_df

## DISCARD CANDIDATES WITHOUT RELEVANT INFORMATION

In [ ]:
if ONE_EXAMPLE_TEST:
  intersection = np.intersect1d(candidates_df.index, features_df.index)

  filtered_candidates_df = candidates_df.loc[intersection]

  filtered_candidates_df = pd.concat([filtered_candidates_df, features_df], axis=1)
  filtered_candidates_df["Class"] = np.zeros_like(filtered_candidates_df.index)
  filtered_candidates_df

## FIND TRUE POSITIVE CANDIDATES

In [ ]:
if ONE_EXAMPLE_TEST:
  neighborhood_th = 12

  hit_indexes, hit_indexes_dict = find_neighborhood_indices(candidates_df, convert_annotation_df(annotations_by_uid(uid, ANNOTATIONS_DF)), neighborhood_th, verbose=True)
  filtered_candidates_df.loc[hit_indexes, "Class"] = np.full((len(hit_indexes)), 1)

  print(f"Hit indexes count: {len(hit_indexes)} --- Positive class count: {len(filtered_candidates_df[filtered_candidates_df['Class'] == 1])} ")

  filtered_candidates_df[filtered_candidates_df["Class"] == 1]

  print(hit_indexes_dict)

[FIND HITS] - Annotations to find:
     x    y    z
8  354  279  121
Annotation with index 8:
x    354
y    279
z    121
Name: 8, dtype: int64


KeyError: "None of [Index([926, 927, 931, 934, 936, 938, 940, 942], dtype='int64')] are in the [index]"

## EXTRACT TO CSV TO TRAIN MODEL

In [ ]:
def get_trainable_candidate_features(uid, subsets_path, annotations_df, neighborhood_th = 12, save_csv_path=None, verbose=False, debug=False):

  # SAFEGUARD NOT TO RUN UNNECESSARY OPERATIONS
  if not len(annotations_by_uid(uid, annotations_df)):
    raise Exception('NO ANNOTATIONS FOR THIS CASE')

  subset = None
  for subset_dir in os.listdir(subsets_path):
    filenames = os.listdir(os.path.join(subsets_path, subset_dir)) # filenames in subset directory
    filenames = list(map(lambda x: os.path.splitext(x)[0], filenames)) # filename w/o extension
    filenames = set(filenames) # unique filenames (because .mhd and .raw have same filename)
    if uid in filenames:
      subset = subset_dir
      break # if found -> stop

  if subset is None:
    raise FileNotFoundError("No image under this uid")

  img_path = os.path.join(subsets_path, subset, f"{uid}.mhd") # .mhd path
  mhd_img = sitk.ReadImage(img_path) # SimpleITK object
  img_3d = sitk.GetArrayFromImage(mhd_img)
  img_3d[img_3d < AIR_TH] = AIR_TH # Normalize minimum intensity value

  print(f"[GET CANDIDATES] - {uid}")
  candidates_df = get_candidates_for_uid(uid, subsets_path, img_3d=img_3d, annotations_df=annotations_df, verbose=verbose)
  print(f"[GET CANDIDATES] - {uid}  - {len(candidates_df)} candidates for case")

  print(f"[GET CANDIDATE FEATURES] - {uid}")
  features_df = extract_features_for_uid(uid, subsets_path, candidates_df, img_3d=img_3d, verbose=verbose, debug=debug)
  print(f"[GET CANDIDATE FEATURES] - {uid}  - {len(features_df)} candidates for case")

  intersection = np.intersect1d(candidates_df.index, features_df.index)

  filtered_candidates_df = candidates_df.loc[intersection]

  filtered_candidates_df = pd.concat([filtered_candidates_df, features_df], axis=1)
  filtered_candidates_df["Class"] = np.zeros_like(filtered_candidates_df.index)

  simple_annotations_df = convert_annotation_df(annotations_by_uid(uid, annotations_df))

  print(f"[GET TRUE POSITIVE CANDIDATES] - {uid}")
  hit_indexes, hit_indexes_dict = find_neighborhood_indices(filtered_candidates_df, simple_annotations_df, neighborhood_th, verbose=verbose)
  filtered_candidates_df.loc[hit_indexes, "Class"] = np.full((len(hit_indexes)), 1)
  print(f"[GET TRUE POSITIVE CANDIDATES] - {uid}  - {len(hit_indexes)} TP candidates for case")

  sensitivity = sensitivity_score(filtered_candidates_df, simple_annotations_df, verbose=verbose)
  print(f"[GET TRUE POSITIVE CANDIDATES] - {uid}  - {sensitivity} sensitivity for")

  if save_csv_path is not None:
    if verbose:
      print(f"[SAVE] - {uid} - candidate features csv")
    # CHANGE THIS LINE FOR FUTURE NOTEBOOKS WITH DIFFERENT PROPERTIES
    save_csv_path = filtered_candidates_df.to_csv(os.path.join(save_csv_path, subset, f"{uid}_candidates.csv"))

  return filtered_candidates_df, (hit_indexes, hit_indexes_dict), sensitivity

## TEST

In [ ]:
print(UIDS)

{'1.3.6.1.4.1.14519.5.2.1.6279.6001.733642690503782454656013446707', '1.3.6.1.4.1.14519.5.2.1.6279.6001.292994770358625142596171316474', '1.3.6.1.4.1.14519.5.2.1.6279.6001.436403998650924660479049012235', '1.3.6.1.4.1.14519.5.2.1.6279.6001.265570697208310960298668720669', '1.3.6.1.4.1.14519.5.2.1.6279.6001.199261544234308780356714831537', '1.3.6.1.4.1.14519.5.2.1.6279.6001.170706757615202213033480003264', '1.3.6.1.4.1.14519.5.2.1.6279.6001.112767175295249119452142211437', '1.3.6.1.4.1.14519.5.2.1.6279.6001.212608679077007918190529579976', '1.3.6.1.4.1.14519.5.2.1.6279.6001.747803439040091794717626507402', '1.3.6.1.4.1.14519.5.2.1.6279.6001.139577698050713461261415990027', '1.3.6.1.4.1.14519.5.2.1.6279.6001.179683407589764683292800449011', '1.3.6.1.4.1.14519.5.2.1.6279.6001.855232435861303786204450738044', '1.3.6.1.4.1.14519.5.2.1.6279.6001.121805476976020513950614465787', '1.3.6.1.4.1.14519.5.2.1.6279.6001.306140003699110313373771452136', '1.3.6.1.4.1.14519.5.2.1.6279.6001.265780642925

In [ ]:
if ONE_EXAMPLE_TEST:
  uid = "1.3.6.1.4.1.14519.5.2.1.6279.6001.511347030803753100045216493273"

  candidate_features, hit_indexes_tuple, sensitivity = get_trainable_candidate_features(
      uid,
      SUBSETS_PATH,
      ANNOTATIONS_DF,
      neighborhood_th=12,
      save_csv_path=SAVE_FOLDER_PATH,
      verbose=False,
      debug=False
  )

  print(f"sensitivity: {sensitivity}")

  print(f"annotations:")
  print(convert_annotation_df(annotations_by_uid(uid, ANNOTATIONS_DF)))

  print(f"hit_indexes for annotation indexes: {hit_indexes_tuple[1]}")
  print(f"hit_indexes: {hit_indexes_tuple[0]}")


Exception: NO ANNOTATIONS FOR THIS CASE

# RUN FOR ALL CASES IN SUBSET

In [27]:
if ONE_EXAMPLE_TEST:
  neighborhood_th = 12

  result_dict = {}

  for i, uid in enumerate(UIDS):
    result_dict[uid] = {}

    # If there is no annotation for a case skip it
    if not len(annotations_by_uid(uid, ANNOTATIONS_DF)):
      continue

    # Check if feature extraction file already exists
    save_folder_subset_path = os.path.join(SAVE_FOLDER_PATH, SUBSET)
    save_filename =  f"{uid}_candidates.csv"
    features_file_path = os.path.join(save_folder_subset_path, save_filename)
    already_extracted = os.listdir(save_folder_subset_path)

    # If feature_extraction file already exists skip it
    if save_filename in os.listdir(save_folder_subset_path):
      print(f"[SKIP] - {uid}  - features have already been extracted for case")
      print("_____________________________________\n")

      continue

    print(f"[START] - {uid} - processing case")
    candidate_features, hit_indexes, sensitivity = get_trainable_candidate_features(
        uid,
        SUBSETS_PATH,
        ANNOTATIONS_DF,
        neighborhood_th=neighborhood_th,
        save_csv_path=SAVE_FOLDER_PATH,
        verbose=False,
        debug=False
    )


    result_dict[uid]["sensitivity"] = sensitivity
    result_dict[uid]["hit_indexes"] = hit_indexes

    print(f"[DONE] - {uid} - processing case")
    print("\n_____________________________________\n")

    # if read_in_count > 5:
    #   break



In [28]:
result_dict

NameError: name 'result_dict' is not defined

In [ ]:
raise Exception("PLEASE STOP HERE")

# ASYNC RUN

In [ ]:
SUBSET

In [ ]:
import os
from concurrent.futures import ThreadPoolExecutor

def process_uid(uid, annotations_df, subsets_path, save_folder_path, subset, neighborhood_th):
    result = {}
    # If there is no annotation for a case skip it
    if not len(annotations_by_uid(uid, annotations_df)):
        print(f"[SKIP] - {uid} - no annotations for case\n")
        return uid, None

    # Check if feature extraction file already exists
    save_folder_subset_path = os.path.join(save_folder_path, subset)
    save_filename = f"{uid}_candidates.csv"
    features_file_path = os.path.join(save_folder_subset_path, save_filename)
    already_extracted = os.listdir(save_folder_subset_path)

    # If feature_extraction file already exists skip it
    if save_filename in already_extracted:
        print(f"[SKIP] - {uid} - features have already been extracted for case\n")
        print("_____________________________________\n")
        return uid, None

    print(f"[START] - {uid}  - processing case {uid}\n")
    candidate_features, hit_indexes, sensitivity = get_trainable_candidate_features(
        uid,
        subsets_path,
        annotations_df,
        neighborhood_th=neighborhood_th,
        save_csv_path=save_folder_path,
        verbose=False,
        debug=False
    )

    result["sensitivity"] = sensitivity
    result["hit_indexes"] = hit_indexes

    print(f"[DONE] - {uid} - processing case")
    print("\n_____________________________________\n")

    return uid, result

def extract_features_concurrently(uids, annotations_df, subsets_path, save_folder_path, subset, neighborhood_th, max_workers=3):
    result_dict = {}

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(process_uid, uid, annotations_df, subsets_path, save_folder_path, subset, neighborhood_th): uid for uid in uids}

        for future in futures:
            uid, result = future.result()
            if result is not None:
                result_dict[uid] = result

    return result_dict

result_dict = extract_features_concurrently(UIDS, ANNOTATIONS_DF, SUBSETS_PATH, SAVE_FOLDER_PATH, SUBSET, neighborhood_th, max_workers=5)
